In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl 
import matplotlib.pyplot as plt

import json
import os
import re

from plotting import *

%matplotlib inline

## Performance over time (evaluation steps) - comparison across $D_{i \in \{Subj, Subj \cup Squad\}}$

In [ ]:
# specify general hyperparameters
models = ['adversarial', 'linear', 'highway', 'recurrent', 'recurrent_highway']

metrics = ['batch_loss', 'batch_acc', 'batch_f1', 'val_f1', 'val_acc', 'val_loss']

task='QA'
version = 'train'
task_setting ='single'
iv = 'datasets'
aux = 'aux_1'

In [ ]:
plotting(models=models, metrics=metrics, task=task, version=version, task_setting=task_setting, iv=iv, aux=aux)

## Performance over time (evaluation steps) - comparison across $\tilde f$  \textit{ (i.e., models)}

In [ ]:
# specify general hyperparameters
models = ['combined', 'SubjQA']

metrics = ['batch_loss', 'batch_acc', 'batch_f1', 'val_f1', 'val_acc', 'val_loss']

task='QA'
version = 'train'
task_setting ='single'
iv = 'models'
aux = 'aux_1'

In [ ]:
plotting(models=models, metrics=metrics, task=task, version=version, task_setting=task_setting, iv=iv, aux=aux)

In [ ]:
# specify general hyperparameters
models = ['combined', 'SubjQA']

metrics = ['batch_loss', 'batch_acc', 'batch_f1', 'val_f1', 'val_acc', 'val_loss']

task='QA'
version = 'train'
task_setting ='multi'
iv = 'models'
aux = 'aux_1'

In [ ]:
plotting(models=models, metrics=metrics, task=task, version=version, task_setting=task_setting, iv=iv, aux=aux)

## \textit{Correlation plots}: performance on dev set as a function of performance on train set

In [ ]:
task = 'QA'
version = 'train'
model = 'highway'
task_setting = 'single'
iv = 'datasets'
metric = ''

all_results = get_results(task=task, version=version, model=model, task_setting=task_setting, aux=None)
plot_results(all_results, task=task, metric=metric, iv=iv,  model=model, task_setting=task_setting, aux=None, correlation=True)

In [ ]:
task = 'QA'
version = 'train'
model = 'linear'
task_setting = 'single'
iv = 'datasets'
metric = ''

all_results = get_results(task=task, version=version, model=model, task_setting=task_setting, aux=None)
plot_results(all_results,task=task, metric=metric, iv=iv,  model=model, task_setting=task_setting, aux=None, correlation=True)

In [ ]:
task = 'QA'
version = 'train'
model = 'recurrent'
task_setting = 'single'
iv = 'datasets'
metric = ''

all_results = get_results(task=task, version=version, model=model, task_setting=task_setting, aux=None)
plot_results(all_results,task=task, metric=metric, iv=iv,  model=model, task_setting=task_setting, aux=None, correlation=True)

In [ ]:
task = 'QA'
version = 'train'
model = 'recurrent_highway'
task_setting = 'single'
iv = 'datasets'
metric = ''

all_results = get_results(task=task, version=version, model=model, task_setting=task_setting, aux=None)
plot_results(all_results,task=task, metric=metric, iv=iv,  model=model, task_setting=task_setting, aux=None, correlation=True)

In [ ]:
task = 'QA'
version = 'train'
model = 'adversarial'
task_setting = 'multi'
iv = 'methods'
metric = ''
aux = 'aux_1'

all_results = get_results(task=task, version=version, model=model, task_setting=task_setting, aux=aux)
plot_results(all_results, task=task, metric=metric, iv=iv,  model=model, task_setting=task_setting, aux=aux, correlation=True)